In [ ]:
import pandas as pd
import numpy as np
import pymongo                                                  
from dotenv import load_dotenv
import os
import time
from datetime import datetime
import nba_api_functions_hist as hist
import nba_prop_functions_hist as prop_hist
import random
import uuid
import threading
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression

In [ ]:
# import data

In [ ]:
regression_results_list = []
by_class_results_list = []

In [ ]:
# try it including both the line point total and the season average point total
X = pts_results_df[['hit_pct_season', 'pts_season_avg_y']]
y = pts_results_df['hit_over']

In [ ]:
# # for column in columns:
# column = 'pts_season_avg'
# df = pts_results_df[pts_results_df['line'] == column]
# X = df[['line_value', 'pts_season_avg_y']]
# y = df['hit_over']
# test_desc = f'{column}_w_both'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Choose a model
model = LogisticRegression()

# 4. Train the model
model.fit(X_train, y_train)


# 5. Evaluate the model
y_pred = model.predict(X_test)
print(model.predict_proba(X_test))
pred_array = model.predict_proba(X_test)
# Adjust this threshold as needed
threshold = 0.9
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba > threshold).astype(int)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report for more detailed metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))

class_1_count = sum(y_pred == 1)
class_0_count = sum(y_pred == 0)
pct_class_1 = round(class_1_count / (class_1_count + class_0_count),2)

print("Number of samples classified as 1:", class_1_count)
print("Number of samples classified as 0:", class_0_count)
print("Percent of samples classified as 1:", pct_class_1)

# If you used Logistic Regression, you can access coefficients
if isinstance(model, LogisticRegression):
    coefficients = model.coef_
    coefficients = coefficients.tolist()
    intercept = model.intercept_
    intercept = intercept.tolist()
    print("Coefficients:", coefficients)
    print("Intercept:", intercept)

# parsing results and storing in dict/datafeed
report_str = classification_report(y_test, y_pred)
report_lines = report_str.split('\n')

items_list = []
for line in report_lines:
    line_items = line.split(' ')
    line_items = [x for x in line_items if x != '']
    items_list.append(line_items)

class_0 = items_list[2][0]
prec_0 = items_list[2][1]
recall_0 = items_list[2][2]
f1_0 = items_list[2][3]
support_0 = items_list[2][4]
class_1 = items_list[3][0]
prec_1 = items_list[3][1]
recall_1 = items_list[3][2]
f1_1 = items_list[3][3]
support_1 = items_list[3][4]
accuracy = items_list[5][1]
total_support = items_list[5][2]

reg_results_dict = {
    'test_desc':test_desc, 'threshold':threshold, 'accuracy':accuracy, 'class_0_precision':prec_0, 'class_1_precision':prec_1,
    'class_0_recall':recall_0, 'class_1_recall':recall_1, 'class_0_f1':f1_0, 'class_1_f1':f1_1,'class_0_support':support_0, 
    'class_1_support':support_1, 'class_0_pred_count':class_0_count, 'class_1_pred_count':class_1_count, 'pct_pred_1':pct_class_1,
    'intercept':intercept, 'coefficients':coefficients
}

class_0_results_dict = {
    'test_desc':test_desc, 'class':0, 'threshold':threshold, 'accuracy':accuracy, 'precision':prec_0,
    'recall':recall_0, 'f1':f1_0, 'support':support_0, 'pred_count':class_0_count, 'pct_pred':(1 - pct_class_1),
    'intercept':intercept, 'coefficients':coefficients
}

class_1_results_dict = {
    'test_desc':test_desc, 'class':1, 'threshold':threshold, 'accuracy':accuracy, 'precision':prec_1,
    'recall':recall_1, 'f1':f1_1, 'support':support_1, 'pred_count':class_1_count, 'pct_pred':pct_class_1,
    'intercept':intercept, 'coefficients':coefficients
}

by_class_results_list.append(class_0_results_dict)
by_class_results_list.append(class_1_results_dict)
regression_results_list.append(reg_results_dict)


regression_df = pd.DataFrame(regression_results_list)
by_class_reg_df = pd.DataFrame(by_class_results_list)